# 오디오 생성

In [1]:
from dotenv import load_dotenv
load_dotenv() 

True

In [2]:
from openai import OpenAI
client = OpenAI()

### 모델로 부터 audio 출력

In [3]:
import warnings
warnings.filterwarnings('ignore')
import base64

Model = "gpt-4o-mini-audio-preview"

completion = client.chat.completions.create(
    model=Model,
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": "진도개는 주인에게 충성스러운가요?"
        }
    ]
)

wav_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open("output/dog.wav", "wb") as f:
    f.write(wav_bytes)

In [4]:
import IPython.display as ipd
ipd.Audio("output/dog.wav")

### Audio를 model에 입력하고 Text 와 Audio 를 동시에 출력

In [5]:
import base64
import requests

# Fetch the audio file and convert it to a base64 encoded string
url = "https://openaiassets.blob.core.windows.net/$web/API/docs/audio/alloy.wav"
response = requests.get(url)
response.raise_for_status()    # HTTP 에러 발생 시 예외 발생
wav_data = response.content
encoded_string = base64.b64encode(wav_data).decode('utf-8')

completion = client.chat.completions.create(
    model="gpt-4o-audio-preview",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": [
                { 
                    "type": "text",
                    "text": "이 녹음파일에는 어떤 내용이 들어 있나요?"
                },
                {
                    "type": "input_audio",
                    "input_audio": {
                        "data": encoded_string,
                        "format": "wav"
                    }
                }
            ]
        },
    ]
)

wav_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open("output/audio_out.wav", "wb") as f:
    f.write(wav_bytes)

In [6]:
completion.choices[0].message.audio.transcript

'이 녹음파일에는 태양이 동쪽에서 뜨고 서쪽으로 진다는 사실을 언급하는 내용이 담겨 있어요. 이건 아주 오랫동안 사람들이 관찰해 온 간단한 자연 현상에 대해 말하고 있습니다. 특별한 설명이나 세부 정보는 없고, 단순한 자연의 사실을 말하는 내용입니다.'

In [7]:
import IPython.display as ipd
ipd.Audio("https://openaiassets.blob.core.windows.net/$web/API/docs/audio/alloy.wav")